In [1]:
import pandas as pd 
from elasticsearch import Elasticsearch
from elasticsearch import helpers

# Paths

In [2]:
img_path = "/home/etd2a/dataset/images/"
data_path = "../parser/final_out.csv"

# Load the data

In [3]:
data = pd.read_csv(data_path)
data.head()

,images,paths,caption,path1,path2,name1,name2
0,image 1: a view of a mountain range from the g...,"cp ""./images/029599.jpg"" vis/imgs/img1.jpg\t",a view of a mountain range from the ground,"""./images/029599.jpg""",vis/imgs/img1.jpg\t,029599.jpg,img1.jpg
1,image 2: a person on a boat in the water\t,"cp ""./images/013369.jpg"" vis/imgs/img2.jpg\t",a person on a boat in the water,"""./images/013369.jpg""",vis/imgs/img2.jpg\t,013369.jpg,img2.jpg
2,image 3: a view of a building with a clock tow...,"cp ""./images/023620.jpg"" vis/imgs/img3.jpg\t",a view of a building with a clock tower,"""./images/023620.jpg""",vis/imgs/img3.jpg\t,023620.jpg,img3.jpg
3,image 4: a street sign on the side of a road\t,"cp ""./images/004139.jpg"" vis/imgs/img4.jpg\t",a street sign on the side of a road,"""./images/004139.jpg""",vis/imgs/img4.jpg\t,004139.jpg,img4.jpg
4,image 5: a man in a suit and tie standing in a...,"cp ""./images/036728.jpg"" vis/imgs/img5.jpg\t",a man in a suit and tie standing in a parking lot,"""./images/036728.jpg""",vis/imgs/img5.jpg\t,036728.jpg,img5.jpg


# Changing the shape

In [4]:
data = data[["caption", "path1", "path2","name1", "name2"]]
data.head()

,caption,path1,path2,name1,name2
0,a view of a mountain range from the ground,"""./images/029599.jpg""",vis/imgs/img1.jpg\t,029599.jpg,img1.jpg
1,a person on a boat in the water,"""./images/013369.jpg""",vis/imgs/img2.jpg\t,013369.jpg,img2.jpg
2,a view of a building with a clock tower,"""./images/023620.jpg""",vis/imgs/img3.jpg\t,023620.jpg,img3.jpg
3,a street sign on the side of a road,"""./images/004139.jpg""",vis/imgs/img4.jpg\t,004139.jpg,img4.jpg
4,a man in a suit and tie standing in a parking lot,"""./images/036728.jpg""",vis/imgs/img5.jpg\t,036728.jpg,img5.jpg


In [5]:
documents = data.to_dict(orient='index')

In [6]:
documents[12999]

{'caption': 'a building with a clock on the side of it',
 'name1': '016238.jpg',
 'name2': 'img13000.jpg',
 'path1': '"./images/016238.jpg"',
 'path2': 'vis/imgs/img13000.jpg\t'}

# Indexing

In [7]:
es = Elasticsearch()

In [8]:
def create_index(index_name, data,type_name="integer"):
    # creates index from a python dict like {0:{key:val}, 1:...}
    
    INDEX_NAME = index_name
    TYPE_NAME = type_name
    
    # filling bulk_data list with two dicts, one contains documents, the other
    # has indexes of those documents
    bulk_data = [] 
    for index, doc in documents.items():
        data_dict = {}
        data_dict = doc
        op_dict = {
            "index": {
                "_index": INDEX_NAME, 
                "_type": TYPE_NAME, 
                "_id": int(str(index)),
            }
        }
        bulk_data.append(op_dict)
        bulk_data.append(data_dict)
        
    # checking if index already exists
    if es.indices.exists(INDEX_NAME):
        print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
    
    # since we are running locally, use one shard and no replicas
    request_body = {
        "settings" : {
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    }
    print("creating '%s' index..." % (INDEX_NAME))
    res = es.indices.create(index = INDEX_NAME, body = request_body)
    print(" response: '%s'" % (res))
    
    # this is where indexing of our data is done
    print("bulk indexing...")
    res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)
#     print(res)
    print("Indexed documents: ", len(res["items"]))

In [9]:
# search = es.search(index = INDEX_NAME, size=2, body={"query": {
#         "match": {
#             "caption": ""
#         }
#     }})
# print(" response: '%s'" % (search))

In [10]:
create_index("images", documents)

deleting 'images' index...
 response: '{'acknowledged': True}'
creating 'images' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'images'}'
bulk indexing...
Indexed documents:  37545
